In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preparing the environment

In [36]:
import os
import sys
import torch
from packaging import version

need_pytorch3d = False
pytorch3dVersion = '0.6.0'

try:
    import pytorch3d as p3d
    if version.parse(p3d.__version__) < version.parse(pytorch3dVersion):
        need_pytorch3d = True
except ModuleNotFoundError:
    need_pytorch3d = True

!curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
!tar xzf 1.10.0.tar.gz
os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [37]:
import pytorch3d as p3d 
print(p3d.__version__)

In [88]:
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
import pytorch3d.structures as py3d_struct

from pytorch3d.io import load_objs_as_meshes, load_obj
from pytorch3d.ops import sample_points_from_meshes

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib

from pytorch3d.renderer import look_at_view_transform, look_at_rotation
from pytorch3d.renderer import OpenGLPerspectiveCameras, FoVPerspectiveCameras, FoVOrthographicCameras
from pytorch3d.renderer import PointLights, DirectionalLights
from pytorch3d.renderer import Materials, RasterizationSettings, PointsRasterizationSettings
from pytorch3d.renderer import MeshRenderer, MeshRasterizer  
from pytorch3d.renderer import SoftPhongShader, HardPhongShader
from pytorch3d.renderer import TexturesUV, TexturesVertex
from pytorch3d.renderer import PointsRenderer, PulsarPointsRenderer, PointsRasterizer
from pytorch3d.renderer import AlphaCompositor, NormWeightedCompositor

sys.path.append(os.path.abspath(''))

In [89]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from PIL import Image
from IPython.display import Image as IPyImage

If you have got a CUDA device, you can use GPU mode

In [90]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')

Downloading the 3D Models

In [91]:
!wget -P Models3D/Geralt https://raw.githubusercontent.com/Aynur19/3D-ML/main/Models3D/Geralt/Geralt.obj
!wget -P Models3D/Geralt https://raw.githubusercontent.com/Aynur19/3D-ML/main/Models3D/Geralt/Geralt.mtl

In [92]:
modelPath = './Models3D/Geralt/Geralt.obj'

# Mesh Rendering

Loading and preparing a 3D model for rendering

In [93]:
def getMesh_Pytorch3D(modelPath: str):
    mesh = load_objs_as_meshes([modelPath], device=device)

    verts = mesh.verts_packed()
    N = verts.shape[0]
    center = verts.mean(0)
    scale = max((verts - center).abs().max(0)[0])
    
    mesh.offset_verts_(-center)
    mesh.scale_verts_((1.0 / float(scale)));

    verts_rgb = torch.ones_like(verts)[None]
    faces = mesh.faces_packed()
    textures = TexturesVertex(verts_features=verts_rgb.to(device))

    mesh = Meshes(
        verts=[verts.to(device)],   
        faces=[faces.to(device)], 
        textures=textures
    )
    
    return mesh

In [94]:
def getMeshRenders_Pytorch3D(mesh, center=((0, 0, 0),), distance=2.5, elevation=0.0, maxAzimuth=360, rangeAzimuth=2):
    frames = []
    
    raster_settings = RasterizationSettings(
        image_size=1024, 
        blur_radius=0.0, 
        faces_per_pixel=1, 
    )

    lights = DirectionalLights(device=device)
    
    for azimuth in range(0, maxAzimuth, rangeAzimuth):
        R, T = look_at_view_transform(distance, elevation, azimuth, at=center) 
        cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

        renderer = MeshRenderer(rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
                                shader=SoftPhongShader(device=device, cameras=cameras, lights=lights))

        frames.append(renderer(mesh)[0, ..., :].cpu().numpy())

    return frames;

Getting render images from different angles

In [95]:
mesh = getMesh_Pytorch3D(modelPath)
frames = getMeshRenders_Pytorch3D(mesh, center=((0, 0, 0),), distance=2.5, elevation=0.0, rangeAzimuth=4)

Getting animation and displaying

In [96]:
def saveGif(frames, path):
    imgFrames = [Image.fromarray((image * 255).astype(np.uint8)) for image in frames]
    meshGif = imgFrames[0]
    meshGif.save(path, format='GIF', append_images=imgFrames, save_all=True, duration=100, loop=0)

In [97]:
meshGifPath = f'./Pytorch3D. Mesh Rendering (Geralt).gif'
saveGif(frames, meshGifPath)

IPyImage(filename=meshGifPath)

# Point Cloud Rendering

In [98]:
points = mesh.verts_packed()
rgb = torch.ones(size=(points.shape[0], 3), device=device)
point_cloud = py3d_struct.Pointclouds(points=[points], features=[rgb])

In [99]:
def getPointCloudRenders_Pytorch3D(pc, center=((0, 0, 0),), distance=2.5, elevation=0.0, maxAzimuth=360, rangeAzimuth=2):
    frames = []
    
    for azimuth in range(0, maxAzimuth, rangeAzimuth):
        R, T = look_at_view_transform(distance, elevation, azimuth)
        cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

        raster_settings = PointsRasterizationSettings(image_size=1024, radius = 0.003, points_per_pixel = 10)
        rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
        renderer = PointsRenderer(rasterizer=rasterizer, compositor=AlphaCompositor())
    
        frames.append(renderer(pc)[0, ..., :].cpu().numpy())
    return frames;

In [100]:
frames = getPointCloudRenders_Pytorch3D(point_cloud, center=((0, 0, 0),), distance=2.5, elevation=0.0, rangeAzimuth=4)

In [101]:
meshGifPath = f'./Pytorch3D. Point Cloud Rendering (Geralt).gif'
saveGif(frames, meshGifPath)

IPyImage(filename=meshGifPath)

# Sampled Point Cloud Rendering

In [102]:
points = sample_points_from_meshes(mesh, 10000)[0]
rgb = torch.ones(size=(points.shape[0], 3), device=device)
point_cloud = py3d_struct.Pointclouds(points=[points], features=[rgb])

In [103]:
frames = getPointCloudRenders_Pytorch3D(point_cloud, center=((0, 0, 0),), distance=2.5, elevation=0.0, rangeAzimuth=4)

In [104]:
meshGifPath = f'./Pytorch3D. Sampled Point Cloud Rendering (Geralt).gif'
saveGif(frames, meshGifPath)

IPyImage(filename=meshGifPath)